# **Jira Release Notes Generator**

This notebook automates the creation of release notes based on data from Jira. It uses OpenAI's GPT models to generate concise, audience-friendly notes and categorize them into predefined sections.

---

### **Options for Data Retrieval**

You can choose to fetch Jira issues using:
1. **CSV Export**: Manually export issues from Jira using the steps below.
2. **API Access**: Automatically retrieve issues from Jira via the API.

---

### **Instructions for Generating the Jira Export (CSV Option)**

1. **Jira Query (JQL):**
   Use the following JQL query to generate the CSV export from Jira:
   ```sql
   project = "GNP" AND labels = release_notes AND fixversion = 12-13-2024 ORDER BY key DESC, created DESC
   ```

2. **Steps to Export:**
   - Navigate to the **Jira Search** page.
   - Enter the above JQL query into the search bar and execute it.
   - Click on the **Export** button (usually located in the top-right corner of the search results).
   - Choose the **CSV** format to download the results.
   - Save the file in a known location (e.g., `Desktop/ReleaseNotes/jira_export.csv`).

---

### **Instructions for Using the API Option**

1. **API Credentials:**
   - Obtain your **Jira API Token**:
     - Go to your Jira account settings > Security > API Tokens > Create and copy a token.
   - Note your Jira instance’s base URL (e.g., `https://yourdomain.atlassian.net`).
   - Ensure your account has API access to the desired project.

2. **Set Up the Notebook:**
   - Update the following variables in the notebook:
     - `JIRA_BASE_URL`: Your Jira instance URL.
     - `JIRA_EMAIL`: Your Jira account email.
     - `JIRA_API_TOKEN`: Your API token.
     - `JQL_QUERY`: The JQL query to fetch issues dynamically.
   - Ensure the `DATA_SOURCE` variable is set to `"api"`.

---

### **How to Run This Notebook**

1. **Set Up Your Environment:**
   - Ensure Python 3 is installed on your system.
   - Create a virtual environment and activate it:
     ```bash
     python3 -m venv venv
     source venv/bin/activate
     ```
   - Install the required dependencies:
     ```bash
     pip install -r requirements.txt
     ```

2. **Prepare Your Files:**
   - If using the CSV option, place the Jira export file in the path specified in the `CSV_FILE_PATH` variable.
   - Ensure the YAML prompt files are located in the `prompts/` directory.

3. **Run the Notebook:**
   - Open the notebook in VSCode or Jupyter Notebook:
     ```bash
     jupyter notebook release_notes_notebook.ipynb
     ```
   - Execute each cell sequentially by clicking the play button (`▶`) or pressing `Shift+Enter`.

4. **Variables to Update:**
   - Set the `DATA_SOURCE` variable to `"csv"` or `"api"` based on your preference.
   - If using the CSV option, update the `CSV_FILE_PATH` variable with the file location.
   - Define the categories for release notes in the `CATEGORIES` variable.

---

### **Output**

The notebook generates:
- A set of concise release notes, formatted in HTML.
- Categorized release notes for easy inclusion in product documentation.

**Example Output Format:**
```html
<p>
  The most recent product release became effective on <strong>July 7th, 2025</strong>.
</p>
<p><strong>General Bug Fixes and Enhancements</strong></p>
<ul>
  <li>Fixed a transparency issue within a dropdown on user home page.</li>
  <li>Restored the functionality of the unordered list button in the rich text editor.</li>
</ul>
<p><strong>Security</strong></p>
<ul>
  <li>Updated user input fields to better sanitize HTML code.</li>
</ul>
```

In [ ]:
# Import Necessary Libraries
import os
import yaml
import requests
import pandas as pd
from datetime import datetime
import openai

In [ ]:
# Set mode: "csv" or "api"
DATA_SOURCE = "api"  # Change to "csv" to load data from a CSV file

# CSV file path (used if DATA_SOURCE = "csv")
CSV_FILE_PATH = "path/to/your/jira_issues.csv"  # Update this path

# JQL query for the issues you want to retrieve (used if DATA_SOURCE = "api")
JQL_QUERY = 'project = "BPD" AND labels = Release_Notes ORDER BY created DESC'

# Categories for release notes
CATEGORIES = ["Performance Improvements", "New Features", "API Changes", "Security Updates"]

# Define Variables
PROMPTS_DIR = "prompts"
CREATE_RELEASE_NOTE_PROMPT_FILE = os.path.join(PROMPTS_DIR, "create_single_sentence_release_note.yaml")
CATEGORIZE_RELEASE_NOTE_PROMPT_FILE = os.path.join(PROMPTS_DIR, "categorize_release_notes.yaml")

# Path to the CSV file containing Jira issues
CSV_FILE_PATH = "path/to/your/jira_issues.csv"  # Update this path if you are using a CSV file

# Get the secrets from the secrets.yaml file
SECRETS_FILE = "supporting_files/secrets.yaml"
with open(SECRETS_FILE, "r") as secrets_file:
    secrets = yaml.safe_load(secrets_file)

# Set the OpenAI API key
openai.api_key = secrets["openai_api_key"]

# Jira API credentials (used if DATA_SOURCE = "api")
jira_base_url = secrets["jira_base_url"] 
jira_email = secrets["jira_email"]
jira_api_token = secrets["jira_api_token"]

In [ ]:
# Load YAML Prompts
def load_yaml_prompt(file_path):
    """Load a YAML prompt from a specified file."""
    with open(file_path, "r") as file:
        return yaml.safe_load(file)

create_release_note_prompt = load_yaml_prompt(CREATE_RELEASE_NOTE_PROMPT_FILE)
categorize_release_notes_prompt = load_yaml_prompt(CATEGORIZE_RELEASE_NOTE_PROMPT_FILE)

In [ ]:
def fetch_issues_from_jira(jql_query, max_results=100):
    """Fetches issues from Jira based on the given JQL query."""
    headers = {
        "Authorization": f"Basic {requests.auth._basic_auth_str(jira_email, jira_api_token)}",
        "Content-Type": "application/json"
    }
    
    params = {
        "jql": jql_query,
        "maxResults": max_results
    }
    
    response = requests.get(f"{jira_base_url}/rest/api/3/search", headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json().get("issues", [])
    else:
        print(f"Failed to fetch issues. Status code: {response.status_code}")
        print(f"Response: {response.text}")
        return []

In [ ]:
# Utility Functions
def get_day_suffix(day):
    """Returns the appropriate suffix for a given day."""
    if 4 <= day <= 20 or 24 <= day <= 30:
        return "th"
    else:
        return ["st", "nd", "rd"][day % 10 - 1]

def get_formatted_date():
    """Returns today's date formatted with day of the week and suffix."""
    today = datetime.today()
    day = today.day
    suffix = get_day_suffix(day)
    month = today.strftime("%B")
    year = today.year
    day_of_week = today.strftime("%A")
    return f"{day_of_week}, {month} {day}{suffix}, {year}"

In [ ]:
# Create Release Note
def create_release_note_for_story(title, description, labels):
    """Generates a single-sentence release note using OpenAI."""
    prompt = create_release_note_prompt["full_prompt"]
    filled_prompt = prompt.replace("{{title}}", title).replace("{{description}}", description).replace("{{labels}}", labels or "No labels provided")
    
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": filled_prompt}
    ]
    
    response = openai.ChatCompletion.create(
        model=create_release_note_prompt["recommended_model"]["model"],
        messages=messages,
        max_tokens=500
    )
    return response["choices"][0]["message"]["content"]

In [ ]:
# Categorize Release Notes
def categorize_release_notes(release_notes, categories):
    """Categorizes release notes using OpenAI."""
    prompt = categorize_release_notes_prompt["full_prompt"]
    filled_prompt = prompt.replace("{{categories}}", ", ".join(categories)).replace("{{release_notes}}", release_notes)
    
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": filled_prompt}
    ]
    
    response = openai.ChatCompletion.create(
        model=categorize_release_notes_prompt["recommended_model"]["model"],
        messages=messages,
        max_tokens=4096
    )
    return response["choices"][0]["message"]["content"]

In [ ]:
# Generate Release Notes
def generate_release_notes_from_csv(csv_file_path, categories):
    """Processes Jira issues from a CSV file and generates categorized release notes."""
    issues_df = pd.read_csv(csv_file_path)
    release_notes = ""
    
    for _, row in issues_df.iterrows():
        title = row.get("Summary", "No Title Provided")
        description = row.get("Description", "No Description Provided")
        labels = row.get("Labels", "No Labels Provided")
        release_note = create_release_note_for_story(title, description, labels)
        release_notes += f"<li>{release_note}</li>\n"
    
    categorized_notes = categorize_release_notes(release_notes, categories)
    formatted_date = get_formatted_date()
    
    return f"""
<p>
  The most recent product release became effective on <strong>{formatted_date}</strong>.
</p>
{categorized_notes}
"""

In [ ]:
# Cell to Run the Code
release_notes_html = generate_release_notes_from_csv(CSV_FILE_PATH, CATEGORIES)
print(release_notes_html)